# 쇼핑몰 지점별 매출액 

## features
- id : 샘플 아이디
- Store : 쇼핑몰 지점 (어디 지역인지는 알 수 없음)
- Date : 주 단위(Weekly) 날짜
- Temperature : 해당 쇼핑몰 주변 기온
- Fuel_Price : 해당 쇼핑몰 주변 연료 가격
- Promotion 1~5 : 해당 쇼핑몰의 비식별화된 프로모션 정보
- Unemployment : 해당 쇼핑몰 지역의 실업률
- IsHoliday : 해당 기간의 공휴일 포함 여부
- Weekly_Sales : 주간 매출액 (목표 예측값)

## 분석 전 예상
1. 주변 평균 기온이 적당(섭씨 18~24도)한 주는 그렇지 않을 때보다 같은 쇼핑몰에서의 주간 매출액이 높았을 것이다 (매출액 상위 10개 지점)
2. 프로모션 수치가 높을 수록 주간 매출액은 높을 것이다 - (비식별화된 프로모션 정보이므로 시각화를 통해 매출과의 연관성 분석)
3. 해당 쇼핑몰의 주변 연료 가격과 주간 매출액과의 연관성은 낮을 것이다 <-> 연료가격이 높아지는 것은 물가가 증가하는 것이므로 전체로 봤을 때 연료 가격이 증가하면 매출액이 적을 것으로 보임(체크)
4. 쇼핑몰 간에 지역의 실업률이 높을수록 매출액이 더 낮을 것이다
<!-- 5. 해당 기간의 공휴일이 있을 경우엔 같은 쇼핑몰일 때 주간 매출액이 높을 것이다 -->

In [ ]:
import pandas as pd
import numpy as np

from scipy.interpolate import make_interp_spline
from plotly.subplots import make_subplots as ms
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly

from datetime import datetime

import os

In [ ]:
file_dir = './data'
file_name = os.path.join(file_dir, 'train.csv')

shop_df = pd.read_csv(file_name).set_index('id')
shop_df.tail(10)

In [ ]:
shop_df.isnull().sum()

### 1. 평균 기온이 적당한 주는 그렇지 않을 때보다 지점이 같은 쇼핑몰에서의 주간 매출액이 높았을 것이다.

온도 범위를 18도 이하 24도 이상 3구간으로 나눠서 분류를 한 column추가

In [ ]:
shop_df['Celcius_temp'] = shop_df['Temperature'].apply(
    lambda x: 'under_10' if ((x-32)/1.8) < 10
                         else ('over_26' if ((x-32)/1.8) > 26 else 'good_temp')
)

shop_df['Fuel_Price'] = shop_df['Fuel_Price'].apply(
    lambda x: round(x,2)
)

shop_df.tail()

In [ ]:
top_rank_store_list = shop_df.groupby('Store').mean()['Weekly_Sales'].sort_values()[::-1][:20].index
top_rank_store_df = pd.DataFrame()

for store_number in top_rank_store_list:
    top_rank_store_df = pd.concat([top_rank_store_df, shop_df.loc[shop_df['Store'] == store_number]])
top_rank_store_df.head()    

In [ ]:
temp_df = top_rank_store_df.groupby(['Store', 'Celcius_temp']).mean().reset_index()
temp_df.head()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('상위 20개 상점의 기온에 따른 거래량')
sns.barplot(data = temp_df, x='Store', y='Weekly_Sales', hue= 'Celcius_temp')
plt.show()

> 기온이 적당하다고 해서 거래량에 직접적인 영향을 준다고 하기엔 어려워보임. -> 계절별 시계열 그래프 찍어서 패턴 찾는 것도 좋아보임  
> 10년 2월부터 12년 9월까지 있으므로 봄은 3~5월, 여름은 6~8월, 가을은 9월~11월 겨울은 12월~2월까지로 함

In [ ]:
shop_df['Date'] = shop_df['Date'].apply(lambda x: '-'.join([x.split('/')[1],x.split('/')[0],x.split('/')[2]]))

In [ ]:
shop_df['Date'] = shop_df['Date'].astype({'Date' : 'datetime64'})
shop_df.head()

In [ ]:
shop_df['Month_Year'] = shop_df["Date"].apply(lambda x: x[3:])

In [ ]:
fig = go.Figure()    
# fig = ms(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x = shop_df.groupby('Date').mean().index, 
                        y = shop_df.groupby('Date').mean()['Weekly_Sales'],
                        mode='lines', 
                        name='Weekly_Sales',)
                        
)

quarter = ['03','06','09','12']
color = ['green', 'red', 'yellow', 'white']
for i in range(len(quarter)):    
    fig.add_shape(type = 'line', x0 = f'2010-{quarter[i]}-01', y0 = 0, x1 = f'2010-{quarter[i]}-01', y1 =1, xref='x', yref='paper',
                  line = {'color' : 'green'})
    fig.add_shape(type = 'rect', x0 = f'2010-{quarter[i]}-01', y0 = 0, x1 = f'2010-{quarter[0] if i//3 == 1 else quarter[i+1]}-01', y1 =1, 
              fillcolor = color[i],opacity = 0.1, line = {'width' : 0}, xref='x',yref='paper')
    
    fig.add_shape(type = 'line', x0 = f'2011-{quarter[i]}-01', y0 = 0, x1 = f'2011-{quarter[i]}-01', y1 =1, xref='x', yref='paper',
                  line = {'color' : 'green'})
    fig.add_shape(type = 'rect', x0 = f'2011-{quarter[i]}-01', y0 = 0, x1 = f'2011-{quarter[0] if i//3 == 1 else quarter[i+1]}-01', y1 =1, 
              fillcolor = color[i],opacity = 0.1, line = {'width' : 0}, xref='x',yref='paper')

    fig.add_shape(type = 'line', x0 = f'2012-{quarter[i]}-01', y0 = 0, x1 = f'2012-{quarter[i]}-01', y1 =1, xref='x', yref='paper',
                  line = {'color' : 'green'})
    fig.add_shape(type = 'rect', x0 = f'2012-{quarter[i]}-01', y0 = 0, x1 = f'2012-{quarter[0] if i//3 == 1 else quarter[i+1]}-01', y1 =1, 
              fillcolor = color[i],opacity = 0.1, line = {'width' : 0}, xref='x',yref='paper')
fig.show()                  

### 2. 프로모션 수치가 높을 수록 주간 매출액은 높을 것이다

In [ ]:
plt.figure(figsize=(15, 10))
corr = sns.heatmap(shop_df.groupby('Store').mean().drop(columns='IsHoliday').corr(method= "spearman"), annot=True, mask = np.triu(shop_df.groupby('Store').mean().drop(columns='IsHoliday').corr(method= "spearman")))
plt.show()

In [ ]:
for i in range(5):
    shop_df[f'Promotion{i+1}'] = shop_df[f'Promotion{i+1}'].fillna(0)
shop_df['sum_of_promotion'] = shop_df[['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']].sum(axis = 1)

In [ ]:
plt.figure(figsize=(15, 10))

sns.regplot(
    data=shop_df.groupby('Store').mean(), x='sum_of_promotion', y="Weekly_Sales",        
)

plt.show()

In [ ]:
fig = ms(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x = shop_df.groupby('Date').mean().index, 
                        y = shop_df.groupby('Date').mean()['Weekly_Sales'],
                        mode='lines', 
                        name='Weekly_Sales',), secondary_y = False
)

fig.add_trace(go.Scatter(x = shop_df.groupby('Date').mean().index, 
                        y = shop_df.groupby('Date').mean()['sum_of_promotion'],
                        mode='lines', 
                        name='sum_of_promotion',), secondary_y = True
)

fig.show() 

> 전체 상점 프로모션의 합의 평균과 주간 판매량의 평균은 상당한 선형관계를 가지고 있는 것으로 보아 높은 관련성이 있는 것을 그래프로 확인 가능

In [ ]:
plt.figure(figsize=(15, 10))
corr = sns.heatmap(shop_df.groupby('Store').mean()[['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5', 'Weekly_Sales']].corr(method= "spearman"), annot=True), #mask = np.triu(shop_df.groupby('Store').mean().drop(columns='IsHoliday').corr(method= "spearman")))
plt.show()

### 3. 해당 쇼핑몰의 주변 연료 가격과 주간 매출액과의 연관성은 낮을 것이다 <-> 연료가격이 높아지는 것은 물가가 증가하는 것이므로 전체로 봤을 때 연료 가격이 증가하면 매출액이 적을 것으로 보임

In [ ]:
top_rank_5_store_list = shop_df.groupby('Store').mean()['Weekly_Sales'].sort_values()[::-1][:5].index
top_rank_5_store_df = pd.DataFrame()

for store_number in top_rank_5_store_list:
    top_rank_5_store_df = pd.concat([top_rank_5_store_df, shop_df.loc[shop_df['Store'] == store_number]])


plt.figure(figsize=(20, 10))

sns.lineplot(
    data=top_rank_5_store_df, x="Fuel_Price", y="Weekly_Sales",
    hue="Store", palette='Set2'
)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(top_rank_5_store_df.groupby('Fuel_Price').mean().index, top_rank_5_store_df.groupby('Fuel_Price').mean()['Weekly_Sales'])
plt.show()

> 기름가격과 판매량 사이의 관계는 크게 없어보이나 그래프가 많이 겹치는 것을 보아 특정 위치에 상관관계가 있어보임, 다만 기름값이 중간을 기점으로 크게 상승할 때엔 전체적으로 Sales_Price가 줄어든 것을 볼 수 있음

In [ ]:
plt.figure(figsize=(25, 10))
sns.lineplot(
    data=top_rank_5_store_df, x="Date", y="Fuel_Price",
    hue="Store", palette='Set2', 
)

plt.show()

> 기름값은 거의 비슷한 추세로 가고 따라서 Date와 Sales_Price와의 연관성을 더 봐야할 것으로 판단

### 4. 쇼핑몰 간 지역 실업률이 높을수록 매출액이 더 낮을 것이다

In [ ]:
for store_num in top_rank_5_store_list:
    # globals()[f'fig_{store_num}'] = go.Figure()    
    globals()[f'fig_{store_num}'] = ms(specs=[[{'secondary_y': True}]])
    
    store_holiday_list = top_rank_5_store_df[(top_rank_5_store_df['IsHoliday'] == True) & (top_rank_5_store_df['Store']== store_num)]['Date'].tolist()
    for i in store_holiday_list:
        # print(i)
        globals()[f'fig_{store_num}'].add_shape(type = 'line', x0 = f'{i}', y0 = 0, x1 = f'{i}', y1 =1, xref='x', yref='paper',
                      line = {'color' : 'green'})
    
    globals()[f'fig_{store_num}'].add_trace(go.Scatter(x = shop_df[shop_df['Store'] == store_num]['Date'], 
                            y = shop_df[shop_df['Store'] == store_num]['Unemployment'],
                            mode='lines', 
                            name=f'Unemployment_of_{store_num}',), secondary_y = False
                            
    )

    globals()[f'fig_{store_num}'].add_trace(go.Scatter(x = shop_df[shop_df['Store'] == store_num]['Date'], 
                            y = shop_df[shop_df['Store'] == store_num]['Weekly_Sales'],
                            mode='lines', 
                            name=f'Weekly_Sales_{store_num}'),secondary_y = True
    )
    

    
    

fig_13.show()
fig_20.show()
fig_14.show()
fig_2.show()
fig_4.show()

> 연관이 없는걸로 보임 하지만 5개의 상점 모두 특정 시기에 판매량이 급증하고 급감하는데 이를 알아볼 필요가 있어보임
- 크리스마스 시즌에 매출이 급증하고 연초에 급감
- 11월 수능시즌에 급증

holiday 찍어보기

공휴일 프로모션 가격 상관성 찍어보기

In [ ]:
corr_df = shop_df.drop(columns=['Store', 'Fuel_Price', 'Temperature'])
corr_df = pd.get_dummies(corr_df, columns = ['IsHoliday', 'Celcius_temp'])

In [ ]:
plt.figure(figsize=(10, 10))
corr = sns.heatmap(corr_df.corr(method= "spearman"), annot=True)
plt.show()

In [ ]:
# for store_num in top_rank_5_store_list:
    
    # 13 , 20 14 , 2 , 4
aaa = top_rank_5_store_df[(top_rank_5_store_df['IsHoliday'] == True) & (top_rank_5_store_df['Store']== 13)]['Date'].tolist()
bbb = top_rank_5_store_df[(top_rank_5_store_df['IsHoliday'] == True) & (top_rank_5_store_df['Store']== 20)]['Date'].tolist()
ccc = top_rank_5_store_df[(top_rank_5_store_df['IsHoliday'] == True) & (top_rank_5_store_df['Store']== 14)]['Date'].tolist()
ddd = top_rank_5_store_df[(top_rank_5_store_df['IsHoliday'] == True) & (top_rank_5_store_df['Store']== 2)]['Date'].tolist()
eee = top_rank_5_store_df[(top_rank_5_store_df['IsHoliday'] == True) & (top_rank_5_store_df['Store']== 4)]['Date'].tolist()
print(aaa)
print(bbb)
print(ccc)
print(ddd)
print(eee)
data = {'13' : aaa,'20': bbb ,'14' : ccc,'2' : ddd,'4' : eee}
pd.DataFrame(data)